<a href="https://colab.research.google.com/github/parkerhale/money/blob/master/CrossLanguageEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross-Language Word Embeddings

We have mentioned, and will discuss in more detail this week, how we can reduce the dimensionality of word representations from their original vectors space to an embedding space on the order of a few hundred dimensions. Different modeling choices for word embeddings may be ultimately evaluated by the effectiveness of classifiers, parsers, and other inference models that use those embeddings.

In this assignment, however, we will consider another common method of evaluating word embeddings: by judging the usefulness of pairwise distances between words in the embedding space.

Follow along with the examples in this notebook, and implement the sections of code flagged with **TODO**.

In [1]:
import gensim
import numpy as np
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity


We'll start by downloading a plain-text version of the Shakespeare plays we used for the first assignment.

In [2]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/shakespeare_plays.txt
lines = [s.split() for s in open('shakespeare_plays.txt')]

--2021-04-17 01:02:22--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/shakespeare_plays.txt
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4746840 (4.5M) [text/plain]
Saving to: ‘shakespeare_plays.txt’

shakespeare_plays.t 100%[===================>]   4.53M  3.16MB/s    in 1.4s    

2021-04-17 01:02:24 (3.16 MB/s) - ‘shakespeare_plays.txt’ saved [4746840/4746840]



Then, we'll estimate a simple word2vec model on the Shakespeare texts.

In [3]:
model = Word2Vec(lines)

Even with such a small training set size, you can perform some standard analogy tasks.

In [4]:
model.wv.most_similar(positive=['king','woman'], negative=['man'])

[('queen', 0.8718737363815308),
 ('prince', 0.7556900382041931),
 ('princess', 0.7553396224975586),
 ('york', 0.7472409605979919),
 ('son', 0.7314462661743164),
 ('margaret', 0.7304731607437134),
 ('duke', 0.7272521257400513),
 ('gloucester', 0.7091763019561768),
 ('emperor', 0.7070452570915222),
 ('sister', 0.70585036277771)]

For the rest of this assignment, we will focus on finding words with similar embeddings, both within and across languages. For example, what words are similar to the name of the title character of *Othello*?

In [5]:
model.wv.most_similar(positive=['othello'])
#model.wv.most_similar(positive=['brutus'])

[('desdemona', 0.9401320815086365),
 ('iago', 0.9354590773582458),
 ('emilia', 0.9342973828315735),
 ('cassio', 0.92005455493927),
 ('imogen', 0.9161376953125),
 ('anne', 0.9118577837944031),
 ('ham', 0.9111126065254211),
 ('pisanio', 0.9106348752975464),
 ('rosalind', 0.9091321229934692),
 ('cressida', 0.9067736268043518)]

This search uses cosine similarity. In the default API, you should see the same similarity between the words `othello` and `desdemona` as in the search results above.

In [6]:
model.wv.similarity('othello', 'desdemona')

0.940132

**TODO**: Your **first task**, therefore, is to implement your own cosine similarity function so that you can reuse it outside of the context of the gensim model object.

In [7]:
model.wv['othello']

array([ 0.01523756, -0.58749044, -0.10889357,  0.21127208,  0.6458807 ,
       -0.10386229, -0.06792642, -0.008529  , -0.9333201 ,  0.5702431 ,
       -0.03729331, -1.1397556 ,  0.6106517 , -0.7049632 ,  0.34696317,
        0.14515096,  0.11001667,  0.32215524,  0.11190315, -0.26259276,
        0.2704374 , -0.20598127, -0.2153087 , -0.01721105, -0.4045657 ,
        0.2550675 ,  0.19392188, -0.37935638,  0.01174349,  0.2714136 ,
       -0.14379409, -0.11775048,  0.18948011,  0.62119913, -0.6950616 ,
        0.22932003,  0.05903176,  0.28678542,  0.2572997 , -0.4439697 ,
       -0.01836766,  0.21567824, -0.23107062,  0.7232963 , -0.57095474,
       -0.20920725, -0.22937194, -0.11639921, -0.77027947,  0.24668284,
       -0.2999118 , -0.20680666,  0.52772593, -0.36610365,  0.45387712,
       -0.7248581 ,  0.11492025, -0.5868485 , -0.27807787,  0.04426297,
        0.18532734, -1.1760302 ,  0.4036734 , -0.07873275,  0.58414173,
        0.07561832,  0.07537149,  0.39330378, -0.11020439, -0.76

In [8]:
## TODO: Implement cosim
def cosim(v1, v2):
  ## return cosine similarity between v1 and v2
  return np.dot(v1, v2)/(np.dot(v1, v1)**0.5*np.dot(v2, v2)**0.5)

## This should give a result similar to model.wv.similarity:
cosim(model.wv['othello'], model.wv['desdemona'])

0.9401320508103402

## Evaluation

We could collect a lot of human judgments about how similar pairs of words, or pairs of Shakespearean characters, are. Then we could compare different word-embedding models by their ability to replicate these human judgments.

If we extend our ambition to multiple languages, however, we can use a word translation task to evaluate word embeddings.

We will use a subset of [Facebook AI's FastText cross-language embeddings](https://fasttext.cc/docs/en/aligned-vectors.html) for several languages. Your task will be to compare English both to French, and to *one more language* from the following set: Arabic, German, Portuguese, Russian, Spanish, Vietnamese, and Chinese.

In [9]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.en.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.fr.vec

# TODO: uncomment at least one of these to work with another language
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.ar.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.de.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.pt.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.ru.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.es.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.vi.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.zh.vec

--2021-04-17 01:02:33--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.en.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67681172 (65M)
Saving to: ‘30k.en.vec’

30k.en.vec          100%[===================>]  64.54M  15.6MB/s    in 5.4s    

2021-04-17 01:02:39 (12.0 MB/s) - ‘30k.en.vec’ saved [67681172/67681172]

--2021-04-17 01:02:39--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.fr.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67802327 (65M)
Saving to: ‘30k.fr.vec’

30k.fr.vec          100%[===================>]  64.66M  16.0MB/s    in 5.3s    

2021-04-17 01:02:45 (12.2 MB/s) - ‘30k.fr.vec’ saved [67802327/67802327]

--2021-04-17 01:02:45--  http://www.ccs.neu.

We'll start by loading the word vectors from their textual file format to a dictionary mapping words to numpy arrays.

In [10]:
def vecref(s):
  (word, srec) = s.split(' ', 1)
  return (word, np.fromstring(srec, sep=' '))

def ftvectors(fname):
  return { k:v for (k, v) in [vecref(s) for s in open(fname)] if len(v) > 1} 

envec = ftvectors('30k.en.vec')
frvec = ftvectors('30k.fr.vec')

# TODO: load vectors for one more language, such as zhvec (Chinese)
# arvec = ftvectors('30k.ar.vec')
devec = ftvectors('30k.de.vec')
# ptvec = ftvectors('30k.pt.vec')
# ruvec = ftvectors('30k.ru.vec')
# esvec = ftvectors('30k.es.vec')
# vivec = ftvectors('30k.vi.vec')
# zhvec = ftvectors('30k.zh.vec')

**TODO**: Your next task is to write a simple function that takes a vector and a dictionary of vectors and finds the most similar item in the dictionary. For this assignment, a linear scan through the dictionary using your `cosim` function from above is acceptible.

In [11]:
## TODO: implement this search function
def mostSimilar(vec, vecDict):
  ## Use your cosim function from above
  mostSimilar = ''
  similarity = 0
  values = list(frvec.values())
  keys = list(frvec.keys())
  for i in range(len(values)):
    if cosim(vec,values[i]) > similarity:
      mostSimilar = keys[i]
      similarity = cosim(vec,values[i])

  return (mostSimilar, similarity)

## some example searches
[mostSimilar(envec[e], frvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']]

[('informatique', 0.5023827767603765),
 ('allemagne', 0.593718413875964),
 ('matrice', 0.5088361302065517),
 ('physique', 0.4555543434796394),
 ('fermentation', 0.3504105196166514)]

Some matches make more sense than others. Note that `computer` most closely matches `informatique`, the French term for *computer science*. If you looked further down the list, you would see `ordinateur`, the term for *computer*. This is one weakness of a focus only on embeddings for word *types* independent of context.

To evalute cross-language embeddings more broadly, we'll look at a dataset of links between Wikipedia articles.

In [12]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/links.tab
links = [s.split() for s in open('links.tab')]

--2021-04-17 01:02:56--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/links.tab
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1408915 (1.3M)
Saving to: ‘links.tab’

links.tab           100%[===================>]   1.34M  1.25MB/s    in 1.1s    

2021-04-17 01:02:58 (1.25 MB/s) - ‘links.tab’ saved [1408915/1408915]



This `links` variable consists of triples of `(English term, language, term in that language)`. For example, here is the link between English `academy` and French `académie`:

In [13]:
links[302]

['academy', 'fr', 'académie']

**TODO**: Evaluate the English and French embeddings by computing the proportion of English Wikipedia articles whose corresponding French article is also the closest word in embedding space. Skip English articles not covered by the word embedding dictionary. Since many articles, e.g., about named entities have the same title in English and French, compute the baseline accuracy achieved by simply echoing the English title as if it were French. Remember to iterate only over English Wikipedia articles, not the entire embedding dictionary.

In [15]:
## TODO: Compute English-French Wikipedia retrieval accuracy.
accuracy = 0
baselineAccuracy = 0
count = 0
correct = 0
for i in range(len(links)):
  EngWord = links[i][0]
  enlist = envec.keys()
  if links[i][1] == 'fr':
    if EngWord in enlist:
      count += 1
      EmbeddingWord = mostSimilar(envec[EngWord], frvec)[0]
      if EmbeddingWord == links[i][2]:
        correct += 1
accuracy = correct/count

count = 0
correct = 0
for i in range(len(links)):
  EngWord = links[i][0]
  enlist = envec.keys()
  if links[i][1] == 'fr':
    if EngWord in enlist:
      count += 1
      if links[i][0] == links[i][2]:
        correct += 1
baselineAccuracy = correct/count
print("fr_accuracy = "+ str(accuracy))
print("fr_baseline accuracy = "+ str(baselineAccuracy))

fr_accuracy = 0.5359205593271862
fr_baseline accuracy = 0.6742324450298915


**TODO**: Compute accuracy and baseline (identity function) acccuracy for Englsih and another language besides French. Although the baseline will be lower for languages not written in the Roman alphabet (i.e., Arabic or Chinese), there are still many articles in those languages with headwords written in Roman characters.

In [16]:
## TODO: Compute English-X Wikipedia retrieval accuracy.
de_accuracy = 0
count = 0
correct = 0
for i in range(len(links)):
  EngWord = links[i][0]
  enlist = envec.keys()
  if links[i][1] == 'de':
    if EngWord in enlist:
      count += 1
      EmbeddingWord = mostSimilar(envec[EngWord], devec)[0]
      if EmbeddingWord == links[i][2]:
        correct += 1
de_accuracy = correct/count
de_baselineAccuracy = 0
count = 0
correct = 0
for i in range(len(links)):
  EngWord = links[i][0]
  enlist = envec.keys()
  if links[i][1] == 'de':
    if EngWord in enlist:
      count += 1
      if links[i][0] == links[i][2]:
        correct += 1
de_baselineAccuracy = correct/count

print("de_accuracy = "+ str(accuracy))
print("de_baseline accuracy = "+ str(baselineAccuracy))

de_accuracy = 0.5359205593271862
de_baseline accuracy = 0.6742324450298915


Further evaluation, if you are interested, could involve looking at the $k$ nearest neighbors of each English term to compute "recall at 10" or "mean reciprocal rank at 10".